In [ ]:
%load_ext autoreload
%autoreload 2

In [91]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, LinearConstraint
from tqdm import tqdm
import torch
import torch.nn as nn
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from src.cstr import *
from src.machinelearning import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
df = generate_training_data(samples=10000, return_df=True)
X = df[['w1', 'w2', 'Cb']].values
y = df['Cb_next'].values

In [93]:
model = SVR(kernel='rbf', C=10).fit(X, y)

In [ ]:
# Tune and train a SVR
from sklearn.model_selection import RandomizedSearchCV

def train_svm(X, y):
    model = SVR()
    param_dist = {
        'C': np.logspace(-3, 1, 100),
        'gamma': np.logspace(-2, 0, 100),
    }
    search = RandomizedSearchCV(model, param_dist, n_iter=25, cv=3, n_jobs=-1, verbose=10)
    search.fit(X, y)
    return search.best_estimator_

model = train_svm(X, y)

In [94]:
# model = NeuralNet(2,1).to(device)
# model = train_nn(model, X, y, 5000)

In [95]:
Cb_ref = np.array([20.9 if t < 20 else 21.0 if t < 40 else 20.5 for t in range(L)])

In [ ]:
Cb_res, w1_res = simulation(model, Cb_ref)

In [ ]:
plot_results(Cb_res, Cb_ref, w1_res)